In [15]:
%load_ext autoreload
%autoreload 2
import numpy as np
from vvasp.VizClasses import NeuropixelsChronicHolder
from vvasp.atlas_utils import Atlas
import pyvista as pv
from labdata.schema import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
insertions2plot = Probe() * ProbeInsertion() & 'subject_name = "MM013"'
insertions = insertions2plot.fetch(format='frame')
insertions.head()

,,,,probe_type,probe_n_shanks,probe_recording_channels,insertion_ap,insertion_ml,insertion_depth,insertion_el,insertion_az,insertion_spin
probe_id,subject_name,procedure_type,procedure_datetime,,,,,,,,,
20097916141,MM013,chronic implant,2023-08-10 13:00:00,0,1,384,-2485.0,-1578.0,3110.0,115.0,-10.0,0.0
20097916762,MM013,chronic implant,2023-08-10 13:00:00,0,1,384,1730.0,-76.0,4510.0,135.0,110.0,0.0
20403312592,MM013,chronic implant,2023-08-10 13:00:00,24,4,384,1313.0,-2221.0,3801.0,85.0,70.0,0.0


In [17]:
#plotter = pv.Plotter(notebook=True)
#plotter.show()

from pyvistaqt import BackgroundPlotter
plotter = BackgroundPlotter()


atlas = Atlas(plotter, min_tree_depth=8, max_tree_depth=8)
atlas.add_atlas_region_mesh('MOp')
atlas.add_atlas_region_mesh('ACAd')
atlas.add_atlas_region_mesh('ACAv')
atlas.add_atlas_region_mesh('VISp')
#atlas.add_atlas_region_mesh('CP')

probes = []
for i, entrypoint in insertions.iterrows():
    entry_point = entrypoint[['insertion_ml','insertion_ap']].values
    depth = entrypoint['insertion_depth']
    angles = entrypoint[['insertion_el','insertion_spin','insertion_az']].values
    if entrypoint.probe_type == '24':
        prb = NeuropixelsChronicHolder('NP24','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
    elif entrypoint.probe_type == '0':
        prb = NeuropixelsChronicHolder('NP1','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
    prb.drive_probe_from_entry(entry_point, angles, depth)
    prb.make_inactive()
    #prb.set_location(np.concatenate([entry_point,np.array([0])]), angles*0)
    probes.append(prb)

plotter.isometric_view()